In [ ]:
# Step 0: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Step 1: Install dependencies and import libraries
!pip install -q transformers
!pip install -q torchinfo
!pip install -q datasets
!pip install -q evaluate
!pip install -q optuna
!pip install -q wandb

import wandb
# Log in to wandb. Replace with your actual WANDB API key
wandb.login(key="your_api_key_here")

# Import DistilBERT tokenizer and model instead of Roberta
from transformers import (DistilBertTokenizer, DistilBertForSequenceClassification, Trainer,
                          TrainingArguments, EarlyStoppingCallback)
from datasets import load_dataset
import numpy as np
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import os


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 18.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.12.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-c

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: peng_zhao (peng_zhao-university-of-california-berkeley) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
# Step 2: Load and preprocess the new dataset (Movies_and_TV)
dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", "raw_review_Movies_and_TV", trust_remote_code=True)

# Shuffle and sample 20% of the full dataset
shuffled_dataset = dataset["full"].shuffle(seed=42)
subset_size = int(0.2 * len(shuffled_dataset))
subset_dataset = shuffled_dataset.select(range(subset_size))

# Filter out samples where rating == 3
subset_dataset = subset_dataset.filter(lambda x: x["rating"] != 3)

# Initialize the DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

# Define tokenization function
def tokenize_and_format(examples):
    outputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=256)
    outputs["labels"] = [1 if rating > 3 else 0 for rating in examples["rating"]]
    return outputs

# Apply the tokenizer
tokenized_dataset = subset_dataset.map(tokenize_and_format, batched=True)

# Save the tokenized dataset to disk
tokenized_dataset.save_to_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_movies_checkpoint")

# Split into training and testing sets
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/30.3k [00:00<?, ?B/s]

Amazon-Reviews-2023.py:   0%|          | 0.00/39.6k [00:00<?, ?B/s]

Movies_and_TV.jsonl:   0%|          | 0.00/8.39G [00:00<?, ?B/s]

Generating full split: 0 examples [00:00, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

Map:   0%|          | 0/3214260 [00:00<?, ? examples/s]

Saving the dataset (0/11 shards):   0%|          | 0/3214260 [00:00<?, ? examples/s]

In [ ]:
# Step 2': Reload dataset from checkpoint if needed
from datasets import load_from_disk
tokenized_dataset = load_from_disk("/content/drive/MyDrive/FP/Checkpoints/tokenized_movies_checkpoint")
tokenized_datasets = tokenized_dataset.train_test_split(test_size=0.2, seed=42)
tokenized_datasets.set_format("torch")


In [ ]:
# Step 3: Load the previously fine-tuned DistilBERT model checkpoint
model = DistilBertForSequenceClassification.from_pretrained(
    "/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_CDs_distilbert", num_labels=2
)

# Compute evaluation metrics
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
    acc = accuracy_score(labels, predictions)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


In [ ]:
# Step 4: Setup TrainingArguments for continued fine-tuning
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/FP/Movies_results_continued",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=192,
    per_device_eval_batch_size=96,
    gradient_accumulation_steps=1,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    fp16=True,
    dataloader_num_workers=6,
    warmup_steps=500,
    report_to=["wandb"],
    run_name="distilbert_movies_and_tv_finetune",  # Update the run name
    logging_steps=50,
    logging_first_step=True
)

# Show dataset sizes
total_train = len(tokenized_datasets["train"])
total_eval = len(tokenized_datasets["test"])
print("Total training samples:", total_train)
print("Total evaluation samples:", total_eval)

# Select a small subset for training/evaluation
train_subset = tokenized_datasets["train"].shuffle(seed=42).select(range(int(0.05 * total_train)))
eval_subset = tokenized_datasets["test"].shuffle(seed=42).select(range(int(0.05 * total_eval)))

# Initialize Trainer for fine-tuning
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=eval_subset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)]
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Total training samples: 2571408
Total evaluation samples: 642852


In [ ]:
# Step 5: Continue fine-tuning on the Movies_and_TV dataset
train_result = trainer.train()

# Print best checkpoint and evaluation metrics
print("Best model checkpoint:", trainer.state.best_model_checkpoint)
print("Best validation metric:", trainer.state.best_metric)
eval_results = trainer.evaluate()
print("Evaluation results of the best model:", eval_results)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.109600,0.102446,0.961328,0.961109,0.960941,0.961328
2,0.075500,0.095900,0.964221,0.963633,0.963463,0.964221
3,0.057800,0.105915,0.964812,0.964817,0.964822,0.964812
4,0.040800,0.119757,0.964283,0.964431,0.964606,0.964283


Best model checkpoint: /content/drive/MyDrive/FP/Movies_results_continued/checkpoint-1340
Best validation metric: 0.09590006619691849


Evaluation results of the best model: {'eval_loss': 0.09590006619691849, 'eval_accuracy': 0.9642212681227055, 'eval_f1': 0.9636334025746888, 'eval_precision': 0.9634629014081372, 'eval_recall': 0.9642212681227055, 'eval_runtime': 30.6546, 'eval_samples_per_second': 1048.522, 'eval_steps_per_second': 10.928, 'epoch': 4.0}


In [ ]:
# Save the final fine-tuned model
trainer.save_model("/content/drive/MyDrive/FP/Checkpoints/final_checkpoint_movies_distilbert")